We will be using an AdaBoost classifier which will be using decision trees as its base (weak learners), we will be using a classification algorithmn since the independent variables/target vairbles are categorical/discrete (since we have edited the dataset).

Importing the necessary libraries :

In [1]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
import sklearn
from sklearn.metrics import accuracy_score
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV

Let's select the datasets we want to work and analyze the data first :

In [3]:
data = pd.read_csv(r"C:\Users\Samraat\Downloads\archive\filtered dataset3.csv")

In [5]:
data1 = pd.read_csv(r"C:\Users\Samraat\Downloads\archive\filtered dataset1.csv")

In [7]:
ave_raisedhands = sum(data['raisedhands'])/len(data['raisedhands'])
ave_VisitedResources = sum(data['VisitedResources'])/len(data['VisitedResources'])
ave_discussion = sum(data['Discussion'])/len(data['Discussion'])
HBL = data.loc[(data['raisedhands'] >= ave_raisedhands) & (data['VisitedResources']>=ave_VisitedResources)  & (data['Class'] == 2)]

In [55]:
print("Students who have above average raised hands and visit the course materials often yet scored low are as follows : ")
print(HBL)

Students who have above average raised hands and visit the course materials often yet scored low are as follows : 
     SectionID  raisedhands  VisitedResources  Discussion  StudentAbsenceDays  \
129          8           70                90           2                   0   
181          8           50                62          33                   1   
444          7           75                72          39                   1   
445          7           72                76          40                   1   

     Class  
129      2  
181      2  
444      2  
445      2  


Now, let us investigate this matter by checking the rest of the features that we had removed :

In [17]:
ave_raisedhands = sum(data1['raisedhands'])/len(data1['raisedhands'])
ave_VisitedResources = sum(data1['VisITedResources'])/len(data1['VisITedResources'])
ave_discussion = sum(data1['Discussion'])/len(data1['Discussion'])
HBL1 = data1.loc[(data1['raisedhands'] >= ave_raisedhands) & (data1['VisITedResources']>=ave_VisitedResources) & (data1['Class'] == 'L')]

In [18]:
print("Students who have above average raised hands and visit the course materials often yet scored low are as follows : ")
print(HBL1)

Students who have above average raised hands and visit the course materials often yet scored low are as follows : 
    SectionID Relation  raisedhands  VisITedResources  AnnouncementsView  \
129         B      one           70                90                  2   
181         B      one           50                62                 13   
444         A      one           75                72                 64   
445         A      one           72                76                 66   

     Discussion ParentAnsweringSurvey ParentschoolSatisfaction  \
129           2                   Yes                     Good   
181          33                    No                      Bad   
444          39                   Yes                     Good   
445          40                   Yes                     Good   

    StudentAbsenceDays Class  Student number  
129            Under-7     L             130  
181            Above-7     L             182  
444            Under-7     L    

Here we can observe that one of the defining features of these students is that all of them have a good relationsip with only one of their parents.

In [21]:
ave_raisedhands = sum(data1['raisedhands'])/len(data1['raisedhands'])
ave_VisitedResources = sum(data1['VisITedResources'])/len(data1['VisITedResources'])
ave_discussion = sum(data1['Discussion'])/len(data1['Discussion'])
HBL1 = data1.loc[(data1['raisedhands'] <= ave_raisedhands) &(data1['Discussion']<=ave_discussion) & (data1['VisITedResources']<=ave_VisitedResources) & (data1['Class'] == 'H')]

In [22]:
print(HBL1)

    SectionID Relation  raisedhands  VisITedResources  AnnouncementsView  \
117         C     both           25                29                 40   
250         A      one           15                 6                 32   
255         A     both           10                17                 12   
273         A     both           22                20                  6   

     Discussion ParentAnsweringSurvey ParentschoolSatisfaction  \
117          17                    No                     Good   
250          40                   Yes                     Good   
255          40                   Yes                     Good   
273          26                    No                      Bad   

    StudentAbsenceDays Class  Student number  
117            Under-7     H             118  
250            Under-7     H             251  
255            Above-7     H             256  
273            Under-7     H             274  


Here we can see that students who have lower than avg raised hands and have not used the provided resources much have still scored good, we can see that the linking features are low absence days and a majorily good relationship with both of their parents. Hence we have been able to obtain a fact that a good relationship with their parents can improve marks. Students who are scoring low despite having high participation and visited resources multiple times can be automatically suggested for counselling to help the students.

Now, let us create the model for the marks predictions, we will use our less complex dataset3 :

In [23]:
X = data.drop(columns=["Class"])
Y = data["Class"]

We have selected the x features to be all but the "Class" feature and the Y feature to be the target variable i.e "Class"

In [24]:
data.head()

,SectionID,raisedhands,VisitedResources,Discussion,StudentAbsenceDays,Class
0,7,15,16,20,0,3
1,7,20,20,25,0,3
2,7,10,7,30,1,2
3,7,30,25,35,1,2
4,7,40,50,50,1,3


In [25]:
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 480 entries, 0 to 479
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype
---  ------              --------------  -----
 0   SectionID           480 non-null    int64
 1   raisedhands         480 non-null    int64
 2   VisitedResources    480 non-null    int64
 3   Discussion          480 non-null    int64
 4   StudentAbsenceDays  480 non-null    int64
 5   Class               480 non-null    int64
dtypes: int64(6)
memory usage: 22.6 KB
None


In [26]:
set1 = np.random.rand(len(data)) < 0.8
train = data[set1]
test = data[~set1]
train_X, test_X, train_y, test_y = train_test_split(X,
    Y, random_state = 1)

Checking and viewing our data,then splitting it into train and test splits with a ratio of 8:2.

In [48]:
classifier = AdaBoostClassifier(
    DecisionTreeClassifier(max_depth = 1), algorithm="SAMME.R",
    n_estimators = 500,learning_rate=1,
    
   
)
classifier.fit(train_X, train_y)

AdaBoostClassifier(estimator=DecisionTreeClassifier(max_depth=1),
                   learning_rate=1, n_estimators=500)

Making the classifier, we will use standard parameters then proceed to find the fit line to train it.

In [28]:
predictions = classifier.predict(test_X)

In [29]:
print(predictions)

[4 4 2 2 4 4 4 2 3 2 2 4 4 3 4 4 2 3 3 2 4 3 3 2 4 4 4 4 2 4 4 3 4 3 4 3 3
 4 4 2 4 4 3 2 3 3 3 4 3 4 3 4 2 2 4 3 2 3 4 4 3 4 2 3 3 4 3 4 3 4 3 3 4 3
 2 4 4 4 2 4 2 2 2 4 4 2 4 3 4 3 3 3 4 3 3 4 2 2 4 2 4 3 4 4 3 4 2 4 4 4 2
 3 4 4 3 3 3 2 4 4]


Here are the initial predictions.

In [49]:
classifier.score(test_X, test_y)

0.675

Upon testing our classifier we can see it has an accuracy of 67.5% , The error margin contains are previously mentioned anomalies as well as actual errors but we will try to minimize the errors anyways.

We will be using gridsearchCV and halvesearchCV to find the best hyperparameters

In [31]:
paramgrid = {'base_estimator__min_samples_split':[6,7,8,9,],
         'base_estimator__min_samples_leaf':[7,8,9],
         'base_estimator__max_depth':[5,6,7],'n_estimators':[100,500,700],'learning_rate':[i for i in range(1,3)]}
R=GridSearchCV(AdaBoostClassifier(base_estimator=DecisionTreeClassifier()),paramgrid,cv = 5,verbose=3,scoring='f1',n_jobs=-1)

R.fit(train_X,train_y)


print(" Results from Grid Search " )
print("\n The best estimator across ALL searched params:\n",R.best_estimator_)
print("\n The best score across ALL searched params:\n",R.best_score_)
print("\n The best parameters across ALL searched params:\n",R.best_params_)

Fitting 5 folds for each of 216 candidates, totalling 1080 fits
 Results from Grid Search 

 The best estimator across ALL searched params:
 AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=5,
                                                         min_samples_leaf=7,
                                                         min_samples_split=6),
                   learning_rate=1, n_estimators=100)

 The best score across ALL searched params:
 nan

 The best parameters across ALL searched params:
 {'base_estimator__max_depth': 5, 'base_estimator__min_samples_leaf': 7, 'base_estimator__min_samples_split': 6, 'learning_rate': 1, 'n_estimators': 100}


C:\Users\Samraat\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:952: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan

In [32]:
paramgrid = {'base_estimator__min_samples_split':[6,7,8,9,],
         'base_estimator__min_samples_leaf':[7,8,9],
         'base_estimator__max_depth':[5,6,7],'n_estimators':[100,500,700],'learning_rate':[i for i in range(1,3)]}
halving_cv = HalvingGridSearchCV(AdaBoostClassifier(base_estimator=DecisionTreeClassifier()), paramgrid, scoring="roc_auc", n_jobs=-1, min_resources="exhaust", factor=3)
halving_cv.fit(train_X,train_y)
# Return set of parameters with the best performance
halving_cv.best_params_

C:\Users\Samraat\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:952: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan

{'base_estimator__max_depth': 5,
 'base_estimator__min_samples_leaf': 7,
 'base_estimator__min_samples_split': 8,
 'learning_rate': 1,
 'n_estimators': 100}

In [50]:
clf1 = AdaBoostClassifier(estimator=DecisionTreeClassifier(max_depth=5,
                                                         min_samples_leaf=7,
                                                         min_samples_split=6),
                   learning_rate=1, n_estimators=100,algorithm="SAMME.R")
clf1.fit(train_X, train_y)

AdaBoostClassifier(estimator=DecisionTreeClassifier(max_depth=5,
                                                    min_samples_leaf=7,
                                                    min_samples_split=6),
                   learning_rate=1, n_estimators=100)

In [52]:
clf2 = AdaBoostClassifier(estimator=DecisionTreeClassifier(max_depth=5,
                                                         min_samples_leaf=7,
                                                         min_samples_split=8),
                   learning_rate=1, n_estimators=100,algorithm="SAMME.R")
clf2.fit(train_X,train_y)

AdaBoostClassifier(estimator=DecisionTreeClassifier(max_depth=5,
                                                    min_samples_leaf=7,
                                                    min_samples_split=8),
                   learning_rate=1, n_estimators=100)

In [51]:
clf1.score(test_X, test_y)



0.7

In [53]:
clf2.score(test_X, test_y)

0.7083333333333334

Now let us take in inputs and try out our machine learning model :

In [54]:
test_x1 = np.asanyarray(test[['SectionID','raisedhands','VisitedResources','Discussion','StudentAbsenceDays']])
test_y1 = np.asanyarray(test[['Class']])

q=int(input("Enter section id : "))
r=int(input("Enter raised hands : "))
j=int(input("Enter VisitedResources : "))
g=int(input("Enter Discussion : "))
h=int(input("Enter StudentAbsenceDays : "))
lvl_of_interest =  np.array([q,r,j,g,h])
lvl_of_interest1 = np.reshape(lvl_of_interest,(1,-1))
testb=clf1.predict(lvl_of_interest1)
if testb==2 :
    print("The predicted score is medium, that is around 33%-50%")
if testb==3 :
    print("The predicted score is medium, that is around 51%-75%")
if testb==4 :
    print("The predicted score is medium, that is around 75%-90%")

Enter section id : 7
Enter raised hands : 59
Enter VisitedResources : 61
Enter Discussion : 90
Enter StudentAbsenceDays : 0
The predicted score is medium, that is around 51%-75%


C:\Users\Samraat\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but AdaBoostClassifier was fitted with feature names
  warnings.warn(


Now we can see that the predicted score is medium, students with low and medium predicted scores can be notified of this prediction providing them with a drive to get better marks than predicted. We can also automatically provide said students with extra course materials and notes. 
With more data features like previous marks and information on their living we can make a better model which will be able to analyze not only student data but also their individual tests and provide them with tips and use linking features to further help them.